# PDF Loader

Hướng dẫn này bao gồm các phương pháp xử lý PDF khác nhau sử dụng LangChain và các thư viện PDF phổ biến.

Xử lý PDF là điều cần thiết để trích xuất và phân tích dữ liệu văn bản từ tài liệu PDF.

Trong hướng dẫn này, chúng ta sẽ khám phá các trình tải (loaders) PDF khác nhau và khả năng của chúng khi làm việc với khung xử lý tài liệu (document processing framework) của LangChain.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## How to load PDFs

[Portable Document Format (PDF)](https://en.wikipedia.org/wiki/PDF), một định dạng tệp được tiêu chuẩn hóa bởi ISO 32000, được Adobe phát triển vào năm 1992 để trình bày tài liệu, bao gồm định dạng văn bản và hình ảnh theo cách độc lập với phần mềm ứng dụng, phần cứng và hệ điều hành.

Hướng dẫn này bao gồm cách tải tài liệu PDF vào định dạng [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) của LangChain. Định dạng này sẽ được sử dụng ở các bước tiếp theo.

LangChain tích hợp với nhiều trình phân tích cú pháp (parsers) PDF khác nhau. Một số đơn giản và tương đối cấp thấp, trong khi những trình khác hỗ trợ OCR và xử lý hình ảnh hoặc thực hiện phân tích bố cục tài liệu nâng cao.

Sự lựa chọn phù hợp phụ thuộc vào ứng dụng của bạn.

Chúng tôi sẽ trình diễn các phương pháp này trên một [tệp mẫu](https://github.com/langchain-ai/langchain/blob/master/libs/community/tests/integration_tests/examples/layout-parser-paper.pdf).
Tải xuống tệp mẫu và sao chép nó vào thư mục dữ liệu của bạn.


In [2]:
FILE_PATH = "data/layout-parser-paper.pdf"

In [3]:
def show_metadata(docs):
    if docs:
        print("[metadata]")
        print(list(docs[0].metadata.keys()))
        print("\n[examples]")
        max_key_length = max(len(k) for k in docs[0].metadata.keys())
        for k, v in docs[0].metadata.items():
            print(f"{k:<{max_key_length}} : {v}")

## PyPDF

[PyPDF](https://github.com/py-pdf/pypdf) là một trong những thư viện Python được sử dụng rộng rãi nhất để xử lý PDF.

Ở đây, chúng ta sử dụng PyPDF để tải PDF dưới dạng danh sách các đối tượng Document.

[`PyPDFLoader`]([https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)) của LangChain tích hợp với PyPDF để phân tích cú pháp các tài liệu PDF thành các đối tượng Document của LangChain.


In [4]:
from langchain_community.document_loaders import PyPDFLoader

# Initialize the PDF loader
loader = PyPDFLoader(FILE_PATH)

# Load data into Document objects
docs = loader.load()

# Print the contents of the document
print(docs[10].page_content[:300])

LayoutParser: A Uniﬁed Toolkit for DL-Based DIA 11
focuses on precision, eﬃciency, and robustness. The target documents may have
complicated structures, and may require training multiple layout detection models
to achieve the optimal accuracy. Light-weight pipelines are built for relatively
simple d


In [5]:
# output metadata
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'author', 'keywords', 'moddate', 'ptex.fullbanner', 'subject', 'title', 'trapped', 'source', 'total_pages', 'page', 'page_label']

[examples]
producer        : pdfTeX-1.40.21
creator         : LaTeX with hyperref
creationdate    : 2021-06-22T01:27:10+00:00
author          : 
keywords        : 
moddate         : 2021-06-22T01:27:10+00:00
ptex.fullbanner : This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2
subject         : 
title           : 
trapped         : /False
source          : data/layout-parser-paper.pdf
total_pages     : 16
page            : 0
page_label      : 1


The `load_and_split()` method allows customizing how documents are chunked by passing a text splitter object, making it more flexible for different use cases.

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load Documents and split into chunks. Chunks are returned as Documents.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = loader.load_and_split(text_splitter=text_splitter)
print(docs[0].page_content)

LayoutParser: A Uniﬁed Toolkit for Deep
Learning Based Document Image Analysis
Zejiang Shen1 (  ), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain
Lee4, Jacob Carlson3, and Weining Li5


### PyPDF(OCR)

Some PDFs contain text images within scanned documents or pictures. You can also use the `rapidocr-onnxruntime` package to extract text from images.

```bash
pip install rapidocr-onnxruntime
```

In [7]:
# Initialize PDF loader, enable image extraction option
loader = PyPDFLoader(FILE_PATH, extract_images=True)

# load PDF page
docs = loader.load()

# access page content
print(docs[4].page_content[:300])

LayoutParser: A Uniﬁed Toolkit for DL-Based DIA 5
Table 1: Current layout detection models in the LayoutParser model zoo
Dataset Base Model1 Large ModelNotes
PubLayNet [38] F / M M Layouts of modern scientiﬁc documents
PRImA [3] M - Layouts of scanned modern magazines and scientiﬁc reports
Newspaper


In [8]:
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'author', 'keywords', 'moddate', 'ptex.fullbanner', 'subject', 'title', 'trapped', 'source', 'total_pages', 'page', 'page_label']

[examples]
producer        : pdfTeX-1.40.21
creator         : LaTeX with hyperref
creationdate    : 2021-06-22T01:27:10+00:00
author          : 
keywords        : 
moddate         : 2021-06-22T01:27:10+00:00
ptex.fullbanner : This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2
subject         : 
title           : 
trapped         : /False
source          : data/layout-parser-paper.pdf
total_pages     : 16
page            : 0
page_label      : 1


### PyPDF Directory

Import all PDF documents from directory.

In [9]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# directory path
loader = PyPDFDirectoryLoader("data")

# load documents
docs = loader.load()

# print the number of documents
docs_len = len(docs)
print(docs_len)

# get document from a directory
document = docs[docs_len - 1]

64


In [10]:
# print the contents of the document
print(document.page_content[:300])

16 Z. Shen et al.
[23] Paszke, A., Gross, S., Chintala, S., Chanan, G., Yang, E., DeVito, Z., Lin, Z.,
Desmaison, A., Antiga, L., Lerer, A.: Automatic diﬀerentiation in pytorch (2017)
[24] Paszke, A., Gross, S., Massa, F., Lerer, A., Bradbury, J., Chanan, G., Killeen,
T., Lin, Z., Gimelshein, N., An


In [11]:
print(document.metadata)

{'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-06-22T01:27:10+00:00', 'author': '', 'keywords': '', 'moddate': '2021-06-22T01:27:10+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data/layout-parser-paper.pdf', 'total_pages': 16, 'page': 15, 'page_label': '16'}


In [12]:
source_set = set()
for doc in docs:
    source_set.add(doc.metadata["source"])

print(source_set)

{'data/01-document-loader-sample.pdf', 'data/layout-parser-paper.pdf'}


## PyMuPDF

[PyMuPDF](https://github.com/pymupdf/PyMuPDF) được tối ưu hóa tốc độ và bao gồm siêu dữ liệu (metadata) chi tiết về PDF và các trang của nó. Nó trả về một document cho mỗi trang.

[`PyMuPDFLoader`]([https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyMuPDFLoader.html](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyMuPDFLoader.html)) của LangChain tích hợp với PyMuPDF để phân tích cú pháp các tài liệu PDF thành các đối tượng Document của LangChain.

```bash
pip install pymupdf
```


In [13]:
from langchain_community.document_loaders import PyMuPDFLoader

# create an instance of the PyMuPDF loader
loader = PyMuPDFLoader(FILE_PATH)

# load the document
docs = loader.load()

# print the contents of the document
print(docs[10].page_content[:300])

LayoutParser: A Uniﬁed Toolkit for DL-Based DIA
11
focuses on precision, eﬃciency, and robustness. The target documents may have
complicated structures, and may require training multiple layout detection models
to achieve the optimal accuracy. Light-weight pipelines are built for relatively
simple d


In [14]:
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'source', 'file_path', 'total_pages', 'format', 'title', 'author', 'subject', 'keywords', 'moddate', 'trapped', 'modDate', 'creationDate', 'page']

[examples]
producer     : pdfTeX-1.40.21
creator      : LaTeX with hyperref
creationdate : 2021-06-22T01:27:10+00:00
source       : data/layout-parser-paper.pdf
file_path    : data/layout-parser-paper.pdf
total_pages  : 16
format       : PDF 1.5
title        : 
author       : 
subject      : 
keywords     : 
moddate      : 2021-06-22T01:27:10+00:00
trapped      : 
modDate      : D:20210622012710Z
creationDate : D:20210622012710Z
page         : 0


In [15]:
len(docs)

16

## Unstructured

[Unstructured](https://docs.unstructured.io/welcome) là một thư viện mạnh mẽ được thiết kế để xử lý nhiều định dạng tài liệu phi cấu trúc và bán cấu trúc khác nhau. Nó vượt trội trong việc tự động xác định và phân loại các thành phần khác nhau trong tài liệu.
Hiện tại hỗ trợ tải các tệp văn bản, PowerPoints, HTML, PDF, hình ảnh và hơn thế nữa.

[`UnstructuredPDFLoader`]([https://python.langchain.com/api_reference/unstructured/document_loaders/langchain_unstructured.document_loaders.UnstructuredLoader.html](https://python.langchain.com/api_reference/unstructured/document_loaders/langchain_unstructured.document_loaders.UnstructuredLoader.html)) của LangChain tích hợp với Unstructured để phân tích cú pháp các tài liệu PDF thành các đối tượng Document của LangChain.

```bash
pip install -qU langchain-community unstructured
pip install pi_heif
pip install unstructured_inference
pip install pdf2image
pip install -U pdfminer.six
pip install pytesseract
pip install unstructured_pytesseract
sudo apt install tesseract-ocr
sudo apt install libtesseract-dev
```


In [16]:
from langchain_community.document_loaders import UnstructuredPDFLoader

# create an instance of UnstructuredPDFLoader
loader = UnstructuredPDFLoader(FILE_PATH)

# load the data
docs = loader.load()

# print the contents of the document
print(docs[0].page_content[:300])

2103.15348v2 [cs.CV] 21 Jun 2021

arXiv

LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis

Zejiang Shen! (4), Ruochen Zhang”, Melissa Dell?, Benjamin Charles Germain Lee*, Jacob Carlson’, and Weining Li>

1 Allen Institute for AI shannons@allenai.org ? Brown University


In [17]:
show_metadata(docs)

[metadata]
['source']

[examples]
source : data/layout-parser-paper.pdf


Về mặt nội bộ, Unstructured tạo ra các "**elements**" khác nhau cho mỗi khối văn bản. Theo mặc định, chúng được kết hợp, nhưng có thể dễ dàng tách biệt bằng cách chỉ định `mode="elements"`.


In [18]:
# Create an instance of UnstructuredPDFLoader (mode="elements”)
loader = UnstructuredPDFLoader(FILE_PATH, mode="elements")

# load the data
docs = loader.load()

# print the contents of the document
print(docs[0].page_content)

2103.15348v2 [cs.CV] 21 Jun 2021


In [19]:
set(doc.metadata["category"] for doc in docs) # extract data categories

{'ListItem', 'NarrativeText', 'Title', 'UncategorizedText'}

In [20]:
show_metadata(docs)

[metadata]
['source', 'coordinates', 'filetype', 'languages', 'last_modified', 'page_number', 'file_directory', 'filename', 'category', 'element_id']

[examples]
source         : data/layout-parser-paper.pdf
coordinates    : {'points': ((51.0, 599.0), (51.0, 1411.0), (96.0, 1411.0), (96.0, 599.0)), 'system': 'PixelSpace', 'layout_width': 1700, 'layout_height': 2200}
filetype       : application/pdf
languages      : ['eng']
last_modified  : 2025-03-11T08:50:59
page_number    : 1
file_directory : data
filename       : layout-parser-paper.pdf
category       : UncategorizedText
element_id     : 717b88dc7738c0e45a97b66c00b98b3d


## PyPDFium2

LangChain's [`PyPDFium2Loader`](
https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFium2Loader.html) integrates with [PyPDFium2](https://github.com/pypdfium2-team/pypdfium2) to parse PDF documents into LangChain Document objects.

In [21]:
from langchain_community.document_loaders import PyPDFium2Loader

# create an instance of the PyPDFium2 loader
loader = PyPDFium2Loader(FILE_PATH)

# load data
docs = loader.load()

# print the contents of the document
print(docs[10].page_content[:300])

LayoutParser: A Unified Toolkit for DL-Based DIA 11
focuses on precision, efficiency, and robustness. The target documents may have
complicated structures, and may require training multiple layout detection models
to achieve the optimal accuracy. Light-weight pipelines are built for relatively
simpl


/home/dino/miniconda3/envs/langchain/lib/python3.10/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


**Lưu ý**: Khi sử dụng `PyPDFium2Loader`, bạn có thể nhận thấy các thông báo cảnh báo liên quan đến `get_text_range()`. Những cảnh báo này là một phần của hoạt động nội bộ của thư viện và không ảnh hưởng đến chức năng xử lý PDF. Bạn có thể tiếp tục an toàn với hướng dẫn mặc dù có những cảnh báo này, vì chúng là một phần bình thường của môi trường phát triển và không ảnh hưởng đến mục tiêu học tập.


In [22]:
show_metadata(docs)

[metadata]
['title', 'author', 'subject', 'keywords', 'creator', 'producer', 'creationdate', 'moddate', 'source', 'total_pages', 'page']

[examples]
title        : 
author       : 
subject      : 
keywords     : 
creator      : LaTeX with hyperref
producer     : pdfTeX-1.40.21
creationdate : 2021-06-22T01:27:10+00:00
moddate      : 2021-06-22T01:27:10+00:00
source       : data/layout-parser-paper.pdf
total_pages  : 16
page         : 0


## PDFMiner
[PDFMiner](https://github.com/pdfminer/pdfminer.six) is a specialized Python library focused on text extraction and layout analysis from PDF documents.

LangChain's [`PDFMinerLoader`](
https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PDFMinerLoader.html) integrates with PDFMiner to parse PDF documents into LangChain Document objects.

In [23]:
from langchain_community.document_loaders import PDFMinerLoader

# Create a PDFMiner loader instance
loader = PDFMinerLoader(FILE_PATH)

# load data
docs = loader.load()

# print the contents of the document
print(docs[0].page_content[:300])

1
2
0
2

n
u
J

1
2

]

V
C
.
s
c
[

2
v
8
4
3
5
1
.
3
0
1
2
:
v
i
X
r
a

LayoutParser: A Uniﬁed Toolkit for Deep
Learning Based Document Image Analysis

Zejiang Shen1 ((cid:0)), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain
Lee4, Jacob Carlson3, and Weining Li5

1 Allen Institute for AI
s


In [24]:
show_metadata(docs)

[metadata]
['author', 'creationdate', 'creator', 'keywords', 'moddate', 'ptex.fullbanner', 'producer', 'subject', 'title', 'trapped', 'total_pages', 'source']

[examples]
author          : 
creationdate    : 2021-06-22T01:27:10+00:00
creator         : LaTeX with hyperref
keywords        : 
moddate         : 2021-06-22T01:27:10+00:00
ptex.fullbanner : This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2
producer        : pdfTeX-1.40.21
subject         : 
title           : 
trapped         : False
total_pages     : 16
source          : data/layout-parser-paper.pdf


### Using PDFMiner to generate HTML text

This method allows you to parse the output HTML content through [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/) to get more structured and richer information about font size, page numbers, PDF header/footer, etc. which can help you semantically split the text into sections.

In [25]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader

# create an instance of PDFMinerPDFasHTMLLoader
loader = PDFMinerPDFasHTMLLoader(FILE_PATH)

# load the document
docs = loader.load()

# print the contents of the document
print(docs[0].page_content[:300])

<html><head>
<meta http-equiv="Content-Type" content="text/html">
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:612px; height:792px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border


In [26]:
show_metadata(docs)

[metadata]
['source']

[examples]
source : data/layout-parser-paper.pdf


# `beautifulsoup4`

```bash
pip install beautifulsoup4
```

In [27]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(docs[0].page_content, "html.parser") # initialize HTML parser
content = soup.find_all("div") # search for all div tags

In [28]:
import re

cur_fs = None
cur_text = ""
snippets = []  # collect all snippets of the same font size
for c in content:
    sp = c.find("span")
    if not sp:
        continue
    st = sp.get("style")
    if not st:
        continue
    fs = re.findall("font-size:(\d+)px", st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text, cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text, cur_fs))
# Note: Possibility to add a strategy for removing duplicate snippets (since the header/footer of a PDF appears across multiple pages, 
# it can be considered duplicate information when found)

In [29]:
from langchain_core.documents import Document

cur_idx = -1
semantic_snippets = []
# Assumption: headings have higher font size than their respective content
for s in snippets:
    # if current snippet's font size > previous section's heading => it is a new heading
    if (
        not semantic_snippets
        or s[1] > semantic_snippets[cur_idx].metadata["heading_font"]
    ):
        metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
        metadata.update(docs[0].metadata)
        semantic_snippets.append(Document(page_content="", metadata=metadata))
        cur_idx += 1
        continue

    # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
    if (
        not semantic_snippets[cur_idx].metadata["content_font"]
        or s[1] <= semantic_snippets[cur_idx].metadata["content_font"]
    ):
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata["content_font"] = max(
            s[1], semantic_snippets[cur_idx].metadata["content_font"]
        )
        continue

    # if current snippet's font size > previous section's content but less than previous section's heading than also make a new
    metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
    metadata.update(docs[0].metadata)
    semantic_snippets.append(Document(page_content="", metadata=metadata))
    cur_idx += 1

print(semantic_snippets[4])

page_content='Recently, various DL models and datasets have been developed for layout analysis
tasks. The dhSegment [22] utilizes fully convolutional networks [20] for segmen-
tation tasks on historical documents. Object detection-based methods like Faster
R-CNN [28] and Mask R-CNN [12] are used for identifying document elements [38]
and detecting tables [30, 26]. Most recently, Graph Neural Networks [29] have also
been used in table detection [27]. However, these models are usually implemented
individually and there is no uniﬁed framework to load and use such models.
There has been a surge of interest in creating open-source tools for document
image processing: a search of document image analysis in Github leads to 5M
relevant code pieces 6; yet most of them rely on traditional rule-based methods
or provide limited functionalities. The closest prior research to our work is the
OCR-D project7, which also tries to build a complete toolkit for DIA. However,
similar to the platform develo

## PDFPlumber

[PDFPlumber](https://github.com/jsvine/pdfplumber) là một thư viện phân tích cú pháp PDF vượt trội trong việc trích xuất văn bản và bảng từ PDF.

[`PDFPlumberLoader`]([https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PDFPlumberLoader.html](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PDFPlumberLoader.html)) của LangChain tích hợp với PDFPlumber để phân tích cú pháp tài liệu PDF thành các đối tượng Document của LangChain.

Giống như PyMuPDF, tài liệu đầu ra chứa siêu dữ liệu (metadata) chi tiết về PDF và các trang của nó, đồng thời trả về một tài liệu cho mỗi trang.

```bash
pip install pdfplumber
```


In [31]:
from langchain_community.document_loaders import PDFPlumberLoader

# create a PDF document loader instance
loader = PDFPlumberLoader(FILE_PATH)

# load the document
docs = loader.load()

# access the first document data
print(docs[10].page_content[:300])

LayoutParser: A Unified Toolkit for DL-Based DIA 11
focuses on precision, efficiency, and robustness. The target documents may have
complicatedstructures,andmayrequiretrainingmultiplelayoutdetectionmodels
to achieve the optimal accuracy. Light-weight pipelines are built for relatively
simple documen


In [32]:
show_metadata(docs)

[metadata]
['source', 'file_path', 'page', 'total_pages', 'Author', 'CreationDate', 'Creator', 'Keywords', 'ModDate', 'PTEX.Fullbanner', 'Producer', 'Subject', 'Title', 'Trapped']

[examples]
source          : data/layout-parser-paper.pdf
file_path       : data/layout-parser-paper.pdf
page            : 0
total_pages     : 16
Author          : 
CreationDate    : D:20210622012710Z
Creator         : LaTeX with hyperref
Keywords        : 
ModDate         : D:20210622012710Z
PTEX.Fullbanner : This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2
Producer        : pdfTeX-1.40.21
Subject         : 
Title           : 
Trapped         : False
